In [ ]:
# pip install chembl_webresource_client

In [67]:
pip install tabulate

Note: you may need to restart the kernel to use updated packages.


## Import packages
The chembl_webresource package should contain the tools to obtain the chEMBL IDs and mechanisms.

In [68]:
from chembl_webresource_client.new_client import new_client
import pandas as pd
import numpy as np
import tabulate

#### Load the list of drugs

In [3]:
druglist_all = pd.read_csv("data/drug_info_GDSC2.csv")

In [90]:
# list_2 = druglist_all["synonyms"].tolist()
# list_2 = [item for sublist in list_2 for item in sublist.split(',')]
# list_2 = [item for item in list_2 if item != '-']


In [93]:
druglist_all['all_names'] = druglist_all['drug_name'] + ', ' + druglist_all['synonyms']

In [151]:
# Split the strings in col3 and create new rows
new_rows = []
for index, row in druglist_all.iterrows():
    elements = row['all_names'].split(',')
    for element in elements:
        new_rows.append([row['drug_id'],  element])

# Create a new DataFrame with the new rows
new_df = pd.DataFrame(new_rows, columns=['drug_id', 'all_names'])

print(new_df)
pd.DataFrame.to_csv(new_df,'output/syn-id_gdsc.csv')

     drug_id        all_names
0       1834           743380
1       1834                -
2       2011      ICL-SIRT078
3       2011                -
4       2038          UNC0638
..       ...              ...
864     1911         Cosmegen
865     1911           Lyovac
866     1911   Meractinomycin
867     1911            Ac-De
868     1911             ActD

[869 rows x 2 columns]


In [108]:
list_1 = new_df["all_names"].tolist()

#### chEMBL IDs retrival
**1st round**, the chembl using the 'names' as 'upper case' they appear in the PRISM database, lookig all synonyms and oprefered names.

In [110]:
n_chemb1 = {}

In [111]:
for i in list_1:
    molecule = new_client.molecule
    mols1 = molecule.filter(pref_name = i.upper()).only('molecule_chembl_id')
    mols2 = molecule.filter(molecule_synonyms__molecule_synonym = i.upper()).only('molecule_chembl_id')
    mols3 = molecule.filter(molecule_synonyms__synonyms = i.upper()).only('molecule_chembl_id')
    if len(mols1) > 0:
        n_chemb1.update({i : mols1[0]["molecule_chembl_id"]})
    elif len(mols2) > 0:
        n_chemb1.update({i : mols2[0]["molecule_chembl_id"]})
    elif len(mols3) > 0:
        n_chemb1.update({i : mols3[0]["molecule_chembl_id"]})
    else:
        n_chemb1.update({i : ":("})

In [112]:
def my_filtering_function(pair):
    unwanted_value = ':('
    key, value = pair
    if value != unwanted_value:
        return False  # filter pair out of the dictionary
    else:
        return True  # keep pair in the filtered dictionary

In [113]:
filtered_chembl1 = dict(filter(my_filtering_function, n_chemb1.items()))

In [114]:
filtered_list1 = list(filtered_chembl1.keys())

In [133]:
len(filtered_list1)

619

**2nd round:** Same but with the original names without transfoming to upper

In [119]:
n_chemb2 = {}

In [120]:
for i in filtered_list1:
    molecule = new_client.molecule
    mols1 = molecule.filter(pref_name = i).only('molecule_chembl_id')
    mols2 = molecule.filter(molecule_synonyms__molecule_synonym = i).only('molecule_chembl_id')
    mols3 = molecule.filter(molecule_synonyms__synonyms = i).only('molecule_chembl_id')
    if len(mols1) > 0:
        n_chemb2.update({i : mols1[0]["molecule_chembl_id"]})
    elif len(mols2) > 0:
        n_chemb2.update({i : mols2[0]["molecule_chembl_id"]})
    elif len(mols3) > 0:
        n_chemb2.update({i : mols3[0]["molecule_chembl_id"]})
    else:
        n_chemb2.update({i : ":("})

In [121]:
filtered_chembl2 = dict(filter(my_filtering_function, n_chemb2.items()))

In [122]:
filtered_list2 = list(filtered_chembl2.keys())

In [123]:
len(filtered_list2)

618

**Round 3**: change presence of  '-'  character

In [124]:
n_chemb3 = {}

In [125]:
for i in filtered_list2:
    molecule = new_client.molecule
    mols1 = molecule.filter(pref_name = i.replace('-', ' ').upper()).only('molecule_chembl_id')
    mols2 = molecule.filter(molecule_synonyms__molecule_synonym = i.replace('-', ' ').upper()).only('molecule_chembl_id')
    mols3 = molecule.filter(molecule_synonyms__synonyms = i.replace('-', ' ').upper()).only('molecule_chembl_id')
    if len(mols1) > 0:
        n_chemb3.update({i : mols1[0]["molecule_chembl_id"]})
    elif len(mols2) > 0:
        n_chemb3.update({i : mols2[0]["molecule_chembl_id"]})
    elif len(mols3) > 0:
        n_chemb3.update({i : mols3[0]["molecule_chembl_id"]})
    else:
        n_chemb3.update({i : ":("})

In [126]:
filtered_chembl3 = dict(filter(my_filtering_function, n_chemb3.items()))

In [127]:
filtered_list3 = list(filtered_chembl3.keys())

In [129]:
len(filtered_list3)

618

**Round 4**: change and add uppercases at the beginning

In [130]:
n_chemb4 = {}
for i in filtered_list3:
    molecule = new_client.molecule
    mols1 = molecule.filter(pref_name = i.title()).only('molecule_chembl_id')
    mols2 = molecule.filter(molecule_synonyms__molecule_synonym = i.title()).only('molecule_chembl_id')
    mols3 = molecule.filter(molecule_synonyms__synonyms = i.title()).only('molecule_chembl_id')
    if len(mols1) > 0:
        n_chemb4.update({i : mols1[0]["molecule_chembl_id"]})
    elif len(mols2) > 0:
        n_chemb4.update({i : mols2[0]["molecule_chembl_id"]})
    elif len(mols3) > 0:
        n_chemb4.update({i : mols3[0]["molecule_chembl_id"]})
    else:
        n_chemb4.update({i : ":("})

In [131]:
filtered_chembl4 = dict(filter(my_filtering_function, n_chemb4.items()))

In [132]:
ln(filtered_chembl4

SyntaxError: unexpected EOF while parsing (1226997624.py, line 1)

## Transform results to work in R

In [134]:
def return_chembls(pair):
    unwanted_value = ':('
    key, value = pair
    if value == unwanted_value:
        return False  # filter pair out of the dictionary
    else:
        return True  # keep pair in the filtered dictionary
 

In [138]:
df_1 = pd.DataFrame.from_dict(
    dict(filter(return_chembls, n_chemb1.items())),orient = 'index')
df_2 = pd.DataFrame.from_dict(
    dict(filter(return_chembls, n_chemb2.items())),orient = 'index')
df_3 = pd.DataFrame.from_dict(
    dict(filter(return_chembls, n_chemb3.items())),orient = 'index')
df_4 = pd.DataFrame.from_dict(
    dict(filter(return_chembls, n_chemb4.items())),orient = 'index')

df_total = pd.concat([df_1,df_2,df_3,df_4])

In [137]:
df_total

,0
Bleomycin,CHEMBL403664
Romidepsin,CHEMBL343448
glutathione,CHEMBL1543
Vincristine,CHEMBL90555
4,CHEMBL4526637
...,...
alpha-lipoic acid,CHEMBL33864
Mirin,CHEMBL570841
Staurosporine,CHEMBL388978
5-Fluorouracil,CHEMBL185


In [140]:
pd.DataFrame.to_csv(df_total,'output/chemblids_df_python_gdsc2.csv')


In [141]:
chembl_id = df_total[0].tolist()

In [72]:
len(np.unique(chembl_id))

1129

In [142]:
MOAs = {}

In [143]:
MOAs = {}
for i in chembl_id:
    mechanism = new_client.mechanism
    mols = mechanism.filter(molecule_chembl_id = i).only('mechanism_of_action')
    l = []
    if len(mols) > 0:
        for j in range(0,len(mols)):
            l.append((mols[j]['mechanism_of_action']))
            s = ', '.join(map(str, l))
            MOAs.update({i : s})
    else:
        MOAs.update({i : ":("})

In [144]:
len(MOAs)

143

In [145]:
definition_chembl = {}
for i in chembl_id:
    molecule = new_client.molecule
    mols = molecule.filter(molecule_chembl_id = i).only('usan_stem_definition')
    if len(mols) > 0:
        definition_chembl.update({i : mols[0]["usan_stem_definition"]})
    else:
        definition_chembl.update({i : ":("})

In [146]:
df_moas_1 = pd.DataFrame.from_dict(MOAs,orient = 'index')
df_moas_2 = pd.DataFrame.from_dict(definition_chembl,orient = 'index')

In [147]:
moas_df  = pd.concat([df_moas_1, df_moas_2], axis=1)

In [150]:
moas_df.columns = ['MOAs','definitions']
moas_df['MOAs'].tolist()

[':(',
 'Histone deacetylase inhibitor',
 ':(',
 ':(',
 ':(',
 ':(',
 'Dual specificity mitogen-activated protein kinase kinase 1 inhibitor, Dual specificity mitogen-activated protein kinase kinase 2 inhibitor',
 'Insulin-like growth factor I receptor inhibitor, Insulin receptor inhibitor',
 'DNA topoisomerase II inhibitor',
 'Estrogen receptor antagonist',
 ':(',
 'Farnesyl diphosphate synthase inhibitor',
 'Rho-associated protein kinase inhibitor, MAP kinase-activated protein kinase 2 inhibitor',
 'DNA inhibitor',
 ':(',
 ':(',
 'DNA inhibitor',
 ':(',
 ':(',
 'DNA inhibitor',
 'DNA inhibitor',
 ':(',
 ':(',
 ':(',
 "Inosine-5'-monophosphate dehydrogenase (IMPDH) inhibitor",
 ':(',
 'Serine/threonine-protein kinase mTOR inhibitor',
 ':(',
 ':(',
 'Glutathione reductase inhibitor, DNA inhibitor, RNA inhibitor',
 'Heat shock protein HSP90 inhibitor',
 ':(',
 ':(',
 'DNA topoisomerase I inhibitor',
 'Serine/threonine-protein kinase Chk1 inhibitor',
 ':(',
 ':(',
 'Serine/threonine-prote

In [149]:
pd.DataFrame.to_csv(moas_df,'output/moas_df_python_gdsc.csv')

In [41]:
list_2 = druglist_all["synonyms"].tolist()

list_2 = [item for sublist in list_2 for item in sublist.split(', ')]
list_2 = [item for item in list_2 if item != '-']

In [43]:
l2_chemb1 = {}

In [44]:
for i in list_2:
    molecule = new_client.molecule
    mols1 = molecule.filter(pref_name = i.upper()).only('molecule_chembl_id')
    mols2 = molecule.filter(molecule_synonyms__molecule_synonym = i.upper()).only('molecule_chembl_id')
    mols3 = molecule.filter(molecule_synonyms__synonyms = i.upper()).only('molecule_chembl_id')
    if len(mols1) > 0:
        l2_chemb1.update({i : mols1[0]["molecule_chembl_id"]})
    elif len(mols2) > 0:
        l2_chemb1.update({i : mols2[0]["molecule_chembl_id"]})
    elif len(mols3) > 0:
        l2_chemb1.update({i : mols3[0]["molecule_chembl_id"]})
    else:
        l2_chemb1.update({i : ":("})

In [45]:
def my_filtering_function(pair):
    unwanted_value = ':('
    key, value = pair
    if value != unwanted_value:
        return False  # filter pair out of the dictionary
    else:
        return True  # keep pair in the filtered dictionary

In [46]:
filtered_chembl1 = dict(filter(my_filtering_function, l2_chemb1.items()))

In [47]:
filtered_list1 = list(filtered_chembl1.keys())

In [49]:
len(filtered_list1)

240

In [50]:
l2_chemb2 = {}

In [51]:
for i in filtered_list1:
    molecule = new_client.molecule
    mols1 = molecule.filter(pref_name = i).only('molecule_chembl_id')
    mols2 = molecule.filter(molecule_synonyms__molecule_synonym = i).only('molecule_chembl_id')
    mols3 = molecule.filter(molecule_synonyms__synonyms = i).only('molecule_chembl_id')
    if len(mols1) > 0:
        l2_chemb2.update({i : mols1[0]["molecule_chembl_id"]})
    elif len(mols2) > 0:
        l2_chemb2.update({i : mols2[0]["molecule_chembl_id"]})
    elif len(mols3) > 0:
        l2_chemb2.update({i : mols3[0]["molecule_chembl_id"]})
    else:
        l2_chemb2.update({i : ":("})

In [53]:
filtered_chembl2 = dict(filter(my_filtering_function, l2_chemb2.items()))

In [1]:
l2_chemb2

NameError: name 'l2_chemb2' is not defined